In [2]:
import concurrent.futures
import os
from pathlib import Path
import cv2
import numpy as np
from tensorflow import keras
from tqdm import tqdm
import sys

2024-05-15 18:21:48.241689: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-15 18:21:48.313839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 18:21:49.657768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Predicting

## Importing the necessary functions

In [26]:
sys.path.append("..") # fix

In [30]:
from prediction.apply_UNET_mask_split import apply_UNET_mask
from prediction.divide_image import divide_image
from prediction.recreate_UNET_image import recreate_UNET_image
from prediction.run import get_image_files

In [35]:
def process_image(index_img, image_file):
    img = cv2.imread(str(Path(image_file_folder, image_file)), 0)
    subdivided_image_raw = divide_image(
        window_size, sub_image_size, img, stride_division
    )

    subdivided_image_UNET = apply_UNET_mask(subdivided_image_raw, UNET_model)
    img_UNET = recreate_UNET_image(
        subdivided_image_UNET, window_size, img, stride_division
    )

    out_img = np.hstack((img, img_UNET))
    cv2.imwrite(str(Path(out_folder, f"img_{index_img:06d}.jpg")), out_img)

## Getting the images

In [36]:
image_file_folder = r"/EXPERIMENTS/HSC_images/22_08_Ql_1_P_1_HIGH_20__001" # 
out_folder = Path("OUTPUT")
out_folder.mkdir(parents=True, exist_ok=True)
image_files = get_image_files(image_file_folder) # This function returns a list which contains the images paths

## Loading the UNET

In [38]:
UNET_model_file = os.path.join("UNET_best.keras")
UNET_model = keras.models.load_model(UNET_model_file, compile=False)
window_size = 128
sub_image_size = 64
stride_division = 8

## Running the UNET

In [34]:
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = []
    for index_img, image_file in enumerate(image_files):
        future = executor.submit(process_image, index_img, image_file)
        futures.append(future)

    for future in tqdm(
        concurrent.futures.as_completed(futures),
        total=len(futures),
        desc="Processing images",
    ):
        future.result()

Processing images:   0%|                               | 0/3200 [00:00<?, ?it/s]

/home/higorem/DeepFlowImaging/UNET/examples/../prediction/recreate_UNET_image.py:40: RuntimeWarning: invalid value encountered in divide
  UNET_image /= UNET_image_nmax
/home/higorem/DeepFlowImaging/UNET/examples/../prediction/recreate_UNET_image.py:43: RuntimeWarning: invalid value encountered in cast
  UNET_image = UNET_image.astype(np.uint8)
Processing images: 100%|████████████████████| 3200/3200 [11:02<00:00,  4.83it/s]
